In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

sim = Aer.get_backend("aer_simulator")
shots = 1024
n_qubits = 3
T = 10

def create_initial_circuit(thetas):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i, theta in enumerate(thetas):
        qc.ry(theta * np.pi, i)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

def apply_feedback(state, mode):
    qc = QuantumCircuit(n_qubits)
    if mode == 'strong':  # 조건 A
        for i, bit in enumerate(state[::-1]):
            if bit == '1':
                qc.x(i)
    elif mode == 'weak':  # 조건 B
        for i, bit in enumerate(state[::-1]):
            if bit == '1' and i % 2 == 0:  # 일부 큐비트만 적용
                qc.x(i)
    elif mode == 'random':  # 조건 C
        bits = np.random.randint(0, 2, n_qubits)
        for i, bit in enumerate(bits):
            if bit == 1:
                qc.x(i)
    qc.barrier()
    return qc

def run(qc):
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    return result.get_counts()

def similarity(counts1, counts2):
    keys = set(counts1) | set(counts2)
    v1 = np.array([counts1.get(k, 0) for k in keys])
    v2 = np.array([counts2.get(k, 0) for k in keys])
    return 1 - cosine(v1, v2)

def run_experiment(mode, thetas):
    history = []
    counts = run(create_initial_circuit(thetas))
    history.append(counts)

    for _ in range(1, T):
        dominant = max(counts, key=counts.get)
        fb_qc = apply_feedback(dominant, mode)
        qc = fb_qc.compose(create_initial_circuit(thetas))
        counts = run(qc)
        history.append(counts)

    # 외부 자극 삽입
    dominant = max(counts, key=counts.get)
    stim_qc = apply_feedback(dominant, mode)
    stim_qc.rx(np.pi, 0)
    stim_qc = stim_qc.compose(create_initial_circuit(thetas))
    counts = run(stim_qc)
    history.append(counts)

    sims = [similarity(history[i], history[i+1]) for i in range(len(history) - 1)]
    return sims

# 공통 초기 상태
thetas = np.random.rand(n_qubits)

# 세 조건 실행ㅌ
sims_strong = run_experiment('strong', thetas)
sims_weak = run_experiment('weak', thetas)
sims_random = run_experiment('random', thetas)

# 결과 시각화
plt.plot(sims_strong, marker='o', label='Strong Feedback (A)')
plt.plot(sims_weak, marker='s', label='Weak Feedback (B)')
plt.plot(sims_random, marker='x', label='Random Feedback (C)')
plt.xlabel("Cycle")
plt.ylabel("Similarity to Previous")
plt.title("Feedback Strength vs Structural Stability")
plt.legend()
plt.grid(True)
plt.show()
